In [2]:
import pandas as pd
import cx_Oracle as cx
from sqlalchemy import create_engine
import chromedriver_autoinstaller
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests
import time

In [240]:
url = 'https://smartstore.naver.com/uniyuni/products/6071556962'

chromedriver_autoinstaller.install()
driver = webdriver.Chrome()
driver.implicitly_wait(5)
driver.get(url)

In [169]:
driver.execute_script("window.scrollTo(0, (document.body.scrollHeight) * 0.15);")

In [170]:
btn = driver.find_elements(By.CSS_SELECTOR, 'a._11xjFby3Le')

In [54]:
soup = BeautifulSoup(driver.page_source)
reviews = soup.select('div._1McWUwk15j')

In [66]:
review_list = []

for i in reviews:
    review_temp = {}

    try:
        review_temp['선택상품'] = i.select('div._2FXNMst_ak')[0].text
    except:
        review_temp['선택상품'] = None

    try:
        review_temp['리뷰작성일'] = i.select('div.iWGqB6S4Lq span._2L3vDiadT9')[0].text
    except:
        review_temp['리뷰작성일'] = None

    try:
        review_temp['별점'] = int(i.select('em._15NU42F3kT')[0].text)
    except:
        review_temp['별점'] = None

    if review_temp['별점'] >= 3:
        review_temp['긍정/부정'] = '긍정'
    elif review_temp['별점'] == None:
        review_temp['긍정/부정'] = None
    else:
        review_temp['긍정/부정'] = '부정'

    try:
        review_temp['리뷰내용'] = i.select('div._1kMfD5ErZ6 span._2L3vDiadT9')[0].text
    except:
        review_temp['리뷰내용'] = None

    review_list.append(review_temp)

In [50]:
def rip_review(x):
    soup = BeautifulSoup(x)
    reviews = soup.select('div._1McWUwk15j')
    review_list = []

    for i in reviews:
        review_temp = {}

        try:
            review_temp['선택상품'] = i.select('div._2FXNMst_ak')[0].text
        except:
            continue

        try:
            review_temp['리뷰작성일'] = i.select('div.iWGqB6S4Lq span._2L3vDiadT9')[0].text
        except:
            review_temp['리뷰작성일'] = None

        try:
            review_temp['별점'] = int(i.select('em._15NU42F3kT')[0].text)
        except:
            review_temp['별점'] = None

        if review_temp['별점'] >= 3:
            review_temp['긍정/부정'] = '긍정'
        elif review_temp['별점'] == None:
            review_temp['긍정/부정'] = None
        else:
            review_temp['긍정/부정'] = '부정'

        try:
            review_temp['리뷰내용'] = i.select('div._1kMfD5ErZ6 span._2L3vDiadT9')[0].text
        except:
            review_temp['리뷰내용'] = None

        review_list.append(review_temp)
    
    return review_list

In [212]:
button_list = driver.find_elements(By.CSS_SELECTOR, 'div._1HJarNZHiI._2UJrM31-Ry a')

In [213]:
button_text = []
for i in button_list:
    button_text.append(i.text)

In [214]:
button_text

['이전', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '다음']

In [215]:
btn = button_list[button_text.index('11')]

In [216]:
btn.click()

In [255]:
class naver_review():
    def __init__(self):
        chromedriver_autoinstaller.install()
        self.driver = webdriver.Chrome()
        self.driver.implicitly_wait(2)

    def connect_url(self, url):
        self.driver.get(url)
        time.sleep(1)
        self.driver.execute_script("window.scrollTo(0, (document.body.scrollHeight)*0.15);")
        time.sleep(0.5)
        self.review_btn = self.driver.find_elements(By.CSS_SELECTOR, 'a._11xjFby3Le')[1]
        self.review_btn.click()
        time.sleep(0.5)

    def rip_review(self, x):
        self.soup = BeautifulSoup(x)
        self.reviews = self.soup.select('div._1McWUwk15j')
        self.review_list = []

        for i in self.reviews:
            self.review_temp = {}

            try:
                self.review_temp['선택상품'] = i.select('div._2FXNMst_ak')[0].text
            except:
                continue

            try:
                self.review_temp['리뷰작성일'] = i.select('div.iWGqB6S4Lq span._2L3vDiadT9')[0].text
            except:
                self.review_temp['리뷰작성일'] = None

            try:
                self.review_temp['별점'] = int(i.select('em._15NU42F3kT')[0].text)
            except:
                self.review_temp['별점'] = None

            if self.review_temp['별점'] >= 3:
                self.review_temp['긍정/부정'] = '긍정'
            elif self.review_temp['별점'] == None:
                self.review_temp['긍정/부정'] = None
            else:
                self.review_temp['긍정/부정'] = '부정'

            try:
                self.review_temp['리뷰내용'] = i.select('div._1kMfD5ErZ6 span._2L3vDiadT9')[0].text
            except:
                self.review_temp['리뷰내용'] = None

            self.review_list.append(self.review_temp)
        
        return self.review_list
    
    def rip_all(self):
        self.all_reviews = []
        self.idx = 1

        while True:
            self.all_reviews += rip_review(self.driver.page_source)
            self.idx += 1

            self.button_list = self.driver.find_elements(By.CSS_SELECTOR, 'div._1HJarNZHiI._2UJrM31-Ry a')
            self.button_text = []
            for i in self.button_list:
                self.button_text.append(i.text)
            try:
                self.btn = self.button_list[self.button_text.index(str(self.idx))]
                self.btn.click()
                time.sleep(0.3)
            except:
                try:
                    self.btn = self.button_list[self.button_text.index('다음')]
                    self.btn.click()
                    time.sleep(0.3)
                except:
                    break

        return self.all_reviews
    
    def page_summary(self):
        summary = {}
        summary['product_name'] = self.driver.find_element(By.CSS_SELECTOR, 'h3._22kNQuEXmb').text
        summary['product_price'] = int(self.driver.find_element(By.CSS_SELECTOR, 'span._1LY7DqCnwR').text.replace(",", ""))
        summary['reviews'] = self.rip_all()

        return summary

In [256]:
test = naver_review()

In [250]:
url = 'https://smartstore.naver.com/uniyuni/products/6071556962'
url2 = 'https://brand.naver.com/applestore/products/9360093290'

In [257]:
test.connect_url(url)

In [258]:
test_dict = test.page_summary()

In [259]:
test_dict

{'product_name': '삼성전자 갤럭시 버즈2 SM-R177',
 'product_price': 99900,
 'reviews': [{'선택상품': '색상: 올리브',
   '리뷰작성일': '23.12.03.',
   '별점': 4,
   '긍정/부정': '긍정',
   '리뷰내용': '배송도 하루 만에 빠르게 도착하고 물건도 좋습니다. 케이스도 함께 3개 시켰는데 케이스와 같이 들어있는 고리? 같은 것이 한개는 누락된 것 같습니다. 후기를 찾아보니 고리가 든 경우도 있고 아닌 경우도 있는 것 같길래 그려러니 하고 쓰려고 합니다. 혹시 그 고리가 필요하신 분은 참고하시길 바라요. 암튼 본품에는 전혀 문제 없고 색상도 예뻐서 마음에 듭니다! 잘 사용할게요!!'},
  {'선택상품': '색상: 올리브',
   '리뷰작성일': '23.12.12.',
   '별점': 5,
   '긍정/부정': '긍정',
   '리뷰내용': '배송빠르고 물건 잘받았어요 \n올리브색상 23년1월 생산제품이구요~\n라이브는 써봤는데 버즈2는 처음이라 터치 적응하면 좋을것같아요~\n케이스는 따로 구매한건데 빼기 귀찮아서...^^잘받았습니다.'},
  {'선택상품': '색상: 라벤더',
   '리뷰작성일': '23.12.12.',
   '별점': 3,
   '긍정/부정': '긍정',
   '리뷰내용': '첨쓰는건데 귀에 꽂았을때 착용감이 기존 이어폰보다 못하네요 한쪽은 잘 떨어지고 제귀가 문제인지 ㅠ'},
  {'선택상품': '색상: 라벤더',
   '리뷰작성일': '23.12.11.',
   '별점': 5,
   '긍정/부정': '긍정',
   '리뷰내용': '너무 예뻐요 마음에는 드는데 예전버즈시리즈 보단 조금 아픈듯..'},
  {'선택상품': '색상: 화이트',
   '리뷰작성일': '23.12.10.',
   '별점': 5,
   '긍정/부정': '긍정',
   '리뷰내용': '잘 받았어요 예상대로 너무 좋네요 ~^^ 다음에 또 주문할게요 ^^'},
  {'선택상품': '색

In [1]:
tex = 'asdfjkasdfkjlhasdfaskljdhfaskjfdf'

In [7]:
n = 6

In [6]:
len(tex) // 5

6

In [8]:
li = []
for i in range(0, n):
    li.append(tex[n*i:n*(i+1)])

In [9]:
li

['asdfjk', 'asdfkj', 'lhasdf', 'askljd', 'hfaskj', 'fdf']

In [15]:
a = ''
for i, v in enumerate(li):
    a += "TO_CLOB('{0}')".format(v)
    if i < len(li)-1:
        a += "||"

In [16]:
a

"TO_CLOB('asdfjk')||TO_CLOB('asdfkj')||TO_CLOB('lhasdf')||TO_CLOB('askljd')||TO_CLOB('hfaskj')||TO_CLOB('fdf')"